Bayan Nezamabad - 20251971, Euan Bourke - 21332142

In [4]:
# Imports
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
import matplotlib.pyplot as plt

In [5]:
# Hyper Params
EPOCHS = 20
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = optimizers.Adam()
VALIDATION_SPLIT=0.90

IMG_ROW, IMG_COL = 224, 224 # Image dimensions
INPUT_SHAPE = (IMG_ROW, IMG_COL, 3) # 3 as RGB
NB_CLASSES = 100 # 100 different classifications for 100 different species of butterfly or moth

In [6]:
class Dataset:
    def __init__(self):
        return
        

In [12]:
class BENet: # Our implementation of AlexNet
    def __init__(self):
        self.model = tf.keras.models.Sequential()
        self.build_layers()
        self.model.compile(optimizer=optimizers.SGD(learning_rate = 0.001), loss="categorical_crossentropy", metrics=['accuracy'])

    def build_layers(self):

        # Convolutional Layers
        self.model.add(layers.Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=INPUT_SHAPE))
        self.model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
        self.model.add(layers.BatchNormalization())

        self.model.add(layers.Conv2D(256, (5, 5), padding='same', activation='relu'))
        self.model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
        self.model.add(layers.BatchNormalization())

        self.model.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
        self.model.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
        self.model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))

        self.model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

        self.model.add(layers.Flatten()) # Flatten Layers

        # Full Connected Layers
        self.model.add(layers.Dropout(rate=0.5))
        self.model.add(layers.Dense(4096, activation='relu'))

        self.model.add(layers.Dropout(rate=0.5))
        self.model.add(layers.Dense(4096, activation='relu'))
        
        self.model.add(layers.Dense(NB_CLASSES, activation='softmax'))
        
        return self.model

model = BENet()

Refs: \
AlexNet Architecture: https://www.kaggle.com/code/blurredmachine/alexnet-architecture-a-complete-guide \
https://medium.com/@siddheshb008/alexnet-architecture-explained-b6240c528bd5
Keras.layer docs: https://www.tensorflow.org/api_docs/python/tf/keras/layers/
